In [1]:
import pandas as pd
import re
from datetime import datetime

path = "/content/drive/MyDrive/ds_portfolio/num_suicide_japan/data"

In [2]:
suicide2015_2016_2017 = pd.read_excel(path+"/01_raw/2015-2017.xlsx")
suicide2018 = pd.read_excel(path+"/01_raw/2018暫定値.xlsx")
suicide2019 = pd.read_excel(path+"/01_raw/2019暫定値.xlsx")
suicide2020 = pd.read_excel(path+"/01_raw/2020zantei.xlsx")
suicide2021 = pd.read_excel(path+"/01_raw/202104zantei.xlsx") # Renew the zantei dataset here by month!

pref = pd.read_excel(path+"/01_raw/pref.xlsx")

# Ken

## 2015-2019

In [3]:
# Wide to Long
suicide2015_2016_2017 = pd.melt(suicide2015_2016_2017, id_vars=suicide2015_2016_2017.columns.values[:3], var_name="month", value_name="num_suicide")
suicide2018 = pd.melt(suicide2018, id_vars=suicide2018.columns.values[:2], var_name="month", value_name="num_suicide")
suicide2019 = pd.melt(suicide2019, id_vars=suicide2019.columns.values[:2], var_name="month", value_name="num_suicide")

# Create day column
suicide2015_2016_2017["day"] = 1
suicide2018["day"] = 1
suicide2019["day"] = 1

# Replace string month with int values
suicide2015_2016_2017["month"] = suicide2015_2016_2017["month"].replace({"1月": 1, "2月": 2, "3月": 3, "4月": 4, "5月": 5, "6月": 6, "7月": 7, "8月": 8, "9月": 9, "10月": 10, "11月": 11, "12月": 12})

# Create date column and drop separated year, month, and day columns
suicide2015_2016_2017["Date"] = pd.to_datetime(suicide2015_2016_2017[['year', 'month', 'day']])
suicide2018["Date"] = pd.to_datetime(suicide2018[['Year', 'month', 'day']])
suicide2019["Date"] = pd.to_datetime(suicide2019[['Year', 'month', 'day']])
suicide2015_2016_2017 = suicide2015_2016_2017.drop(["ken", "year", "month", "day"], axis=1)
suicide2018 = suicide2018.drop(["Year", "month", "day"], axis=1)
suicide2019 = suicide2019.drop(["Year", "month", "day"], axis=1)

# Change the column name of prefecture
suicide2018 = suicide2018.rename({"kenname2": "kenname"}, axis=1)
suicide2019 = suicide2019.rename({"kenname2": "kenname"}, axis=1)

# Concat 2015-2019
suicide2015_2019 = pd.concat([suicide2015_2016_2017, suicide2018, suicide2019], axis=0)

# Add pref_id
suicide2015_2019 = pd.merge(suicide2015_2019, pref[["pref_kanji_w", "pref_id"]], left_on="kenname", right_on="pref_kanji_w", how="left").drop("pref_kanji_w", axis=1)

# Set the index
suicide2015_2019 = suicide2015_2019.set_index("Date")

## 2020

In [4]:
# Extract the data from the raw excel file
suicide2020 = suicide2020[8:55]
suicide2020.columns = ["kenname", "Unnamed: 1", "合計", "1月", "2月", "3月", "4月", "5月", "6月", "7月", "8月", "9月", "10月", "11月", "12月"]
suicide2020 = suicide2020.drop(["Unnamed: 1", "12月", "合計"], axis=1)

# Wide to Long
suicide2020 = pd.melt(suicide2020, id_vars=suicide2020.columns.values[:1], var_name="month", value_name="num_suicide")


# Datetime
suicide2020["year"] = 2020
suicide2020["day"] = 1
suicide2020["month"] = suicide2020["month"].replace({"1月": 1, "2月": 2, "3月": 3, "4月": 4, "5月": 5, "6月": 6, "7月": 7, "8月": 8, "9月": 9, "10月": 10, "11月": 11, "12月": 12})
suicide2020["Date"] = pd.to_datetime(suicide2020[['year', 'month', 'day']])
suicide2020 = suicide2020.drop(["year", "month", "day"], axis=1)

# Drop branks
for i in range(len(suicide2020["kenname"])):
    suicide2020.loc[i, "kenname"] = re.sub("\u3000", "", suicide2020.loc[i, "kenname"])

# Add pref_id
suicide2020 = pd.merge(suicide2020, pref[["pref_kanji", "pref_id"]], left_on="kenname", right_on="pref_kanji", how="left").drop("pref_kanji", axis=1)

# Add "県" or something
suicide2020 = pd.merge(suicide2020, pref[["pref_kanji_w", "pref_id"]], left_on="pref_id", right_on="pref_id", how="left").drop("kenname", axis=1)
suicide2020 = suicide2020.rename({"pref_kanji_w": "kenname"}, axis=1)

# Set the index
suicide2020 = suicide2020.set_index("Date")

## 2021

In [5]:
# Extract the data from the raw excel file
suicide2021 = suicide2021[8:55]
suicide2021.columns = ["kenname", "Unnamed: 1", "合計", "1月", "2月", "3月", "4月", "5月", "6月", "7月", "8月", "9月", "10月", "11月", "12月"]
suicide2021 = suicide2021.drop(["Unnamed: 1", "12月", "合計"], axis=1)

# Wide to Long
suicide2021 = pd.melt(suicide2021, id_vars=suicide2021.columns.values[:1], var_name="month", value_name="num_suicide")

# Datetime
suicide2021["year"] = 2021
suicide2021["day"] = 1
suicide2021["month"] = suicide2021["month"].replace({"1月": 1, "2月": 2, "3月": 3, "4月": 4, "5月": 5, "6月": 6, "7月": 7, "8月": 8, "9月": 9, "10月": 10, "11月": 11, "12月": 12})
suicide2021["Date"] = pd.to_datetime(suicide2021[['year', 'month', 'day']])
suicide2021 = suicide2021.drop(["year", "month", "day"], axis=1)

# Drop branks
for i in range(len(suicide2021["kenname"])):
    suicide2021.loc[i, "kenname"] = re.sub("\u3000", "", suicide2021.loc[i, "kenname"])

# Add pref_id
suicide2021 = pd.merge(suicide2021, pref[["pref_kanji", "pref_id"]], left_on="kenname", right_on="pref_kanji", how="left").drop("pref_kanji", axis=1)

# Add "県" or something
suicide2021 = pd.merge(suicide2021, pref[["pref_kanji_w", "pref_id"]], left_on="pref_id", right_on="pref_id", how="left").drop("kenname", axis=1)
suicide2021 = suicide2021.rename({"pref_kanji_w": "kenname"}, axis=1)

# Set the index
suicide2021 = suicide2021.set_index("Date")

# Drop months not having data
suicide2021 = suicide2021.dropna()

## Concat 2015_2019, 2020, and 2021

In [6]:
# Concat all data
suicide = pd.concat([suicide2015_2019, pd.concat([suicide2020, suicide2021], axis=0)], axis=0)
suicide["num_suicide"] = suicide["num_suicide"].astype("float")

# Adjust the order
suicide = suicide.reset_index().sort_values(by=["pref_id", "Date"],).set_index("Date")
# Store the data
suicide.to_pickle(path+"/02_intermediate/suicide.pickle")

In [7]:
suicide

,kenname,num_suicide,pref_id
Date,,,
2015-01-01,北海道,94.0,1
2015-02-01,北海道,66.0,1
2015-03-01,北海道,111.0,1
2015-04-01,北海道,103.0,1
2015-05-01,北海道,102.0,1
...,...,...,...
2020-11-01,沖縄県,12.0,47
2021-01-01,沖縄県,22.0,47
2021-02-01,沖縄県,14.0,47


# Nationwide

In [8]:
nation = suicide.reset_index().groupby(["Date"]).sum().drop("pref_id", axis=1)

# Add the latest data of 202012 for num of suicide
from pandas.tseries.offsets import DateOffset
nation = nation.reset_index()
a = nation.Date+DateOffset(months=1)
nation202012 = pd.DataFrame({"Date": [a[70]], "num_suicide": [1694]})
nation = pd.concat([nation, nation202012], axis=0)
nation = nation.sort_values("Date")
nation = nation.set_index("Date")

nation["num_suicide"].to_pickle(path+"/02_intermediate/nation_suicide.pickle")

In [9]:
nation

,num_suicide
Date,
2015-01-01,2057.0
2015-02-01,1771.0
2015-03-01,2301.0
2015-04-01,2094.0
2015-05-01,2244.0
...,...
2020-12-01,1694.0
2021-01-01,1729.0
2021-02-01,1678.0
